In [1]:
import numpy as np
import pandas as pd
import math
import heapq
import time
from datetime import datetime, timedelta, date
import holidays
from calendar import isleap

#### Setting start date

In [2]:
# get days in each month
def get_day_dict(year):
    # Base dictionary for days in each month
    day_dict = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
                7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    # Adjust February for leap years
    if isleap(year):
        day_dict[2] = 29
    return day_dict

In [3]:
year = 2024
month = 8
day = 1
start_date = datetime(year, month, day) 

In [4]:
# holidays in Taiwan
tw_holidays = holidays.TW(years=year)
print(tw_holidays)

{datetime.date(2024, 1, 1): '中華民國開國紀念日', datetime.date(2024, 2, 9): '農曆除夕', datetime.date(2024, 2, 10): '春節', datetime.date(2024, 2, 11): '春節', datetime.date(2024, 2, 12): '春節', datetime.date(2024, 2, 28): '和平紀念日', datetime.date(2024, 4, 4): '兒童節; 民族掃墓節', datetime.date(2024, 6, 10): '端午節', datetime.date(2024, 9, 17): '中秋節', datetime.date(2024, 10, 10): '國慶日', datetime.date(2024, 4, 5): '兒童節（補假）', datetime.date(2024, 2, 13): '春節（補假）', datetime.date(2024, 2, 14): '春節（補假）', datetime.date(2024, 2, 8): '休息日（2024-02-17日起取代）'}


#### Setting Order Number

In [5]:
# 8, 10, 12, 14, 16
order_num = 16

#### Creating Set

In [6]:
# --- Calendar & working-day index ---
day_dict = get_day_dict(year) # {month: days_in_month}; Feb handles leap years internally
Num_L = day_dict[month] # number of calendar days in target month

# Set of calendar day indices (0-based) that are non-working (weekends or holidays)
# Example: if Aug 3 is Saturday, '2' will be in this set.
HL = set()
for day in range(1, day_dict[month] + 1):
    current_date = date(year, month, day)
    if current_date.weekday() >= 5 or current_date in tw_holidays:  # Check for weekends or holidays
        HL.add(day - 1) # day index is start from zero

# Map working-day index -> calendar day-of-month (both 0-based)
# Example: if Aug 1 (0) is working, L_date[0] == 0; if Aug 2 is holiday, it is skipped.
L_date = dict()
no = 0 
for l in range(Num_L):
    if l not in HL:
        L_date[no] = l
        no += 1
        
# Working-day index set (e.g., {0,1,2,...} counting only working days)
L = set(L_date.keys())

# --- Orders ---
Num_K = order_num # number of orders
K = set(range(Num_K)) # order index set {0..Num_K-1}

# --- Equipment ---
# Number of machines for machine type
Ed = [10, 10]
# Total number of machines
Num_E = sum(Ed)
# Machine index set: {0, 1, ..., Num_E-1}
E = set(range(Num_E))

# Number of bearings (Machine type 1: 2, Machine type 2: 3)
Pd = [2, 3]
# Total number of bearings 
Num_P = int(np.dot(Ed, Pd))
# Bearing index set: {0, 1, ..., Num_P-1}
P = set(range(Num_P))

# --- Order sides (e.g., A/B) ---
# Number of sides per order (2 → sides A and B)
Num_I = 2
# Side index set: {0, 1}  (map 0 to 'A', 1 to 'B')
I = set(range(Num_I))

#### Creating Parameter

In [7]:
# Reading Order Data
# order quantity
with open(f"./data/order/{order_num}/quantity.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    q = list(map(int, content.split()))

# order value
with open(f"./data/order/{order_num}/value.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    v = list(map(int, content.split()))

# order due date
with open(f"./data/order/{order_num}/due_date.txt", 'r') as file:
    # Read the content of the file and split it 
    content = file.read()
    u = list(map(int, content.split()))
    u = [d - 1 for d in u]  # convert from 1-based day numbers to 0-based indices    

In [8]:
# --- Penalty & processing parameters ---
r = 0.01 # tardiness penalty weight
a = 8  # stage 1 processing duration
b = 100  # stage 2 throughput capacity
M = 9999

#### Creating Equipment Schedule, Order Schedule and Equipment Usage 

In [9]:
# equipment schedule
eq_schedule = dict()    
for p in P:
    eq_schedule[p] = []

# equipment usage tracker
eq_usage = dict()
for l in L:
    eq_usage[l] = set()

# equipment availablility tracker
eq_avail = dict()
for p in P:
    eq_avail[p] = set()

#### Creating Orders

In [10]:
# --- Build order dictionary ---
# Each order k has:
#   - qty: order quantity
#   - due: due date (0-based working-day index)
#   - value: economic or priority value
#   - stage: list of processing stages (side A and B, each with two stages)

orders = {}
for k in K:
    orders[k] = {}
    orders[k]['qty'] = q[k] 
    orders[k]['due'] = u[k]
    orders[k]['value'] = v[k]
    orders[k]['stage'] = []
    for i in I: 
        for j in [0, 1]:
            detail = {}
            detail['id'] = j
            if i == 0:
                detail['side'] = "A"
            if i == 1:
                detail['side'] = "B"
                
            if j == 0:
                detail['process_t'] = a  # Stage 1: fixed duration 'a'
            else:
                detail['process_t'] = math.ceil(q[k] / b) # Stage 2: depends on order quantity (ceil(q/b))
    
            orders[k]['stage'].append(detail)

In [11]:
orders

{0: {'qty': 1900,
  'due': 19,
  'value': 12823,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 19},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 19}]},
 1: {'qty': 1300,
  'due': 14,
  'value': 33358,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 13},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 13}]},
 2: {'qty': 1200,
  'due': 15,
  'value': 27368,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 12},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 12}]},
 3: {'qty': 1400,
  'due': 18,
  'value': 93904,
  'stage': [{'id': 0, 'side': 'A', 'process_t': 8},
   {'id': 1, 'side': 'A', 'process_t': 14},
   {'id': 0, 'side': 'B', 'process_t': 8},
   {'id': 1, 'side': 'B', 'process_t': 14}]},
 4: {'qty': 1900,
  'due': 24,
  'value': 44398,
  'stage': [{'i

#### Sorting Orders

In [12]:
order_seq = []
for k in K:
    qty = orders[k]['qty']  # Use -qty as key because heapq is a min-heap 
    heapq.heappush(order_seq, (-qty, k))

In [13]:
order_seq

[(-2000, 9),
 (-1900, 0),
 (-1900, 13),
 (-1700, 7),
 (-1900, 4),
 (-1300, 11),
 (-1800, 12),
 (-1300, 1),
 (-1200, 8),
 (-1400, 3),
 (-1800, 10),
 (-800, 5),
 (-1200, 2),
 (-1200, 6),
 (-1600, 14),
 (-800, 15)]

#### Scheduling 

In [14]:
# record cpu time
start_time = time.time()

In [15]:
# Check whether bearing `p` is available for the continuous time window [start_t, end_t),
# and if feasible, push it into the candidate equipment heap.

# Feasibility rules:
#   1) Capacity rule: each time slot must have available bearing capacity (< b/2 in use)
#   2) Availability rule: bearing `p` must not already be occupied in that window

def _check_aval(p, start_t, end_t, cand_eq):
    # 1) remaining capacity check across the whole window
    for t_p in range(start_t, end_t):
        if len(eq_usage[t_p]) >= b/2: # per-time-slot capacity limit for concurrent bearing usage
            return cand_eq
            
    # 2) bearing availability check
    for t_p in range(start_t, end_t):
        if t_p in eq_avail[p]:
            return cand_eq

    heapq.heappush(cand_eq, (start_t, p))
    return cand_eq

In [16]:
# --- Dispatching rule: pop next order by priority and schedule its stages ----

while order_seq:
    _, k = heapq.heappop(order_seq)
    for task in orders[k]['stage']:
        # --- Stage 1 (window of length a) ---
        if task['id'] == 0:
            # Collect feasible candidates as (start_time, bearing) in a min-heap by start_time
            cand_equip = []
            
            # Try every bearing p and every start slot l where the full window [l, l+a) fits
            for p in P:
                for l in range(len(L) - a + 1):
                     cand_equip = _check_aval(p, l, l + a , cand_equip)
    
            # Choose the start time slot with min start time
            if len(cand_equip) > 0:
                st, br = heapq.heappop(cand_equip)
                # Record scheduled block: (order_id, side, stage_id, start_t, end_t)
                # Stage 1 occupies a contiguous window of length a
                eq_schedule[br].append((k, task['side'], task['id'], st, st + a - 1))
                
                # Update equipment usage (bearing) 
                # Update equipment availability (time)
                for l in range(st, st + a):
                    eq_usage[l].add(br)
                    eq_avail[br].add(l)

                # Keep Stage 1 completion time to gate Stage 2
                end_time_s1 = st + a - 1
                
            else:
                print("!!! [stage 1] no feasible start window found")

        # --- Stage 2 (unit-length slots) ----
        if task['id'] == 1:
            # Each unit of Stage 2 processing consumes exactly one time slot
            for _ in range(task['process_t']):
                cand_equip = []
                
                # Try every bearing p and every time l strictly after Stage 1 ends
                for p in P:
                    for l in range(len(L)):
                        # Stage 2 must start after Stage 1
                        if l > end_time_s1: 
                            cand_equip = _check_aval(p, l, l + 1, cand_equip)

                if len(cand_equip) > 0:
                    st, br = heapq.heappop(cand_equip)
                    
                    # Record a single-slot assignment for Stage 2
                    eq_schedule[br].append((k, task['side'], task['id'], st, st))
                    
                    # Update equipment usage (equipment, bearing) 
                    # Update equipment availability (time)
                    eq_usage[st].add(br)
                    eq_avail[br].add(st)
                                
                else:
                    print("!!! [stage 2] no feasible slot found")
    
                

In [17]:
# End the timer 
cpu_time = time.time() - start_time

#### Calculate Objective Function

In [18]:
# Create a dictionary to store order end time
order_max_end_time = dict()
for p in P:
    for order_id, side, stage, start_t, end_t in eq_schedule[p]:
        if stage == 1:
            if order_id not in order_max_end_time:
                order_max_end_time[order_id] = 0
            if L_date[end_t] > order_max_end_time[order_id]:
                order_max_end_time[order_id] = L_date[end_t]

In [19]:
# Compute tardiness for each order
order_tard = {}
for order_id, max_end_time in order_max_end_time.items():
    tard = max(max_end_time - u[order_id], 0)
    order_tard[order_id] = tard

{9: 0, 0: 0, 4: 0, 13: 0, 12: 0, 7: 0, 3: 2, 11: 0, 6: 6, 5: 5, 10: 0, 14: 0, 15: 2, 1: 6, 8: 0, 2: 6}


In [20]:
# Compute week tardiness and objective
obj = 0 
order_tard_week = {}
for key, value in order_tard.items():
    order_tard_week[key] = math.ceil(value / 7)
    obj += v[key] * order_tard_week[key] * r

In [21]:
print(obj)
print(f"cpu time: {cpu_time:.2f} seconds")

3380.89
cpu time: 0.14 seconds


#### Generating Excel

In [22]:
# --- Create DataFrame structure for the schedule visualization ---
tuple_list = []
p = 0 # global bearing index (incremented sequentially across all lines)


# Iterate over each machine type
for d in range(len(Ed)): 
    offset = sum(Ed[:d]) # starting machine index for this machine type
    for e_local in range(Ed[d]): 
        e_global = offset + e_local
        for _ in range(Pd[d]):   
            tuple_list.append((f"Equipment {e_local}", f"Bearing {p}" ))
            p+=1
                

col_name = pd.MultiIndex.from_tuples(tuple_list, names=["Equipment", "Bearing"])

# Create DataFrame for all calendar days (not just working days)
df = pd.DataFrame(
    index=[f"Day {day}" for day in range(Num_L)],
    columns=col_name
)

In [23]:
df

Equipment Equipment 0           Equipment 1           Equipment 2            \
Bearing     Bearing 0 Bearing 1   Bearing 2 Bearing 3   Bearing 4 Bearing 5   
Day 0             NaN       NaN         NaN       NaN         NaN       NaN   
Day 1             NaN       NaN         NaN       NaN         NaN       NaN   
Day 2             NaN       NaN         NaN       NaN         NaN       NaN   
Day 3             NaN       NaN         NaN       NaN         NaN       NaN   
Day 4             NaN       NaN         NaN       NaN         NaN       NaN   
Day 5             NaN       NaN         NaN       NaN         NaN       NaN   
Day 6             NaN       NaN         NaN       NaN         NaN       NaN   
Day 7             NaN       NaN         NaN       NaN         NaN       NaN   
Day 8             NaN       NaN         NaN       NaN         NaN       NaN   
Day 9             NaN       NaN         NaN       NaN         NaN       NaN   
Day 10            NaN       NaN         NaN       NaN         NaN       NaN   
Day 11            NaN       NaN         NaN       NaN         NaN       NaN   
Day 12            NaN       NaN         NaN       NaN         NaN       NaN   
Day 13            NaN       NaN         NaN       NaN         NaN       NaN   
Day 14            NaN       NaN         NaN       NaN         NaN       NaN   
Day 15            NaN       NaN         NaN       NaN         NaN       NaN   
Day 16            NaN       NaN         NaN       NaN         NaN       NaN   
Day 17            NaN       NaN         NaN       NaN         NaN       NaN   
Day 18            NaN       NaN         NaN       NaN         NaN       NaN   
Day 19            NaN       NaN         NaN       NaN         NaN       NaN   
Day 20            NaN       NaN         NaN       NaN         NaN       NaN   
Day 21            NaN       NaN         NaN       NaN         NaN       NaN   
Day 22            NaN       NaN         NaN       NaN         NaN       NaN   
Day 23            NaN       NaN         NaN       NaN         NaN       NaN   
Day 24            NaN       NaN         NaN       NaN         NaN       NaN   
Day 25            NaN       NaN         NaN       NaN         NaN       NaN   
Day 26            NaN       NaN         NaN       NaN         NaN       NaN   
Day 27            NaN       NaN         NaN       NaN         NaN       NaN   
Day 28            NaN       NaN         NaN       NaN         NaN       NaN   
Day 29            NaN       NaN         NaN       NaN         NaN       NaN   
Day 30            NaN       NaN         NaN       NaN         NaN       NaN   

Equipment Equipment 3           Equipment 4            ... Equipment 6  \
Bearing     Bearing 6 Bearing 7   Bearing 8 Bearing 9  ...  Bearing 40   
Day 0             NaN       NaN         NaN       NaN  ...         NaN   
Day 1             NaN       NaN         NaN       NaN  ...         NaN   
Day 2             NaN       NaN         NaN       NaN  ...         NaN   
Day 3             NaN       NaN         NaN       NaN  ...         NaN   
Day 4             NaN       NaN         NaN       NaN  ...         NaN   
Day 5             NaN       NaN         NaN       NaN  ...         NaN   
Day 6             NaN       NaN         NaN       NaN  ...         NaN   
Day 7             NaN       NaN         NaN       NaN  ...         NaN   
Day 8             NaN       NaN         NaN       NaN  ...         NaN   
Day 9             NaN       NaN         NaN       NaN  ...         NaN   
Day 10            NaN       NaN         NaN       NaN  ...         NaN   
Day 11            NaN       NaN         NaN       NaN  ...         NaN   
Day 12            NaN       NaN         NaN       NaN  ...         NaN   
Day 13            NaN       NaN         NaN       NaN  ...         NaN   
Day 14            NaN       NaN         NaN       NaN  ...         NaN   
Day 15            NaN       NaN         NaN       NaN  ...         NaN   
Day 16            NaN       NaN         NaN       NaN  ...  

In [24]:
# Fill Scheduling DataFrame
for p in P:
    for k, side, stage, start_t, end_t in eq_schedule[p]:
        for l in range(start_t, end_t + 1):                                                
            col_mask = df.columns.get_level_values(1) == f"Bearing {p}" # identify columns where the 2nd MultiIndex level == "Bearing {p}"
            df.loc[f"Day {L_date[l]}", col_mask] = f"Order {k} Side {side} Stage {stage+1}"

In [25]:
df

Equipment              Equipment 0                           \
Bearing                  Bearing 0                Bearing 1   
Day 0       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 1       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 2                          NaN                      NaN   
Day 3                          NaN                      NaN   
Day 4       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 5       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 6       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 7       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 8       Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 9                          NaN                      NaN   
Day 10                         NaN                      NaN   
Day 11      Order 9 Side A Stage 1   Order 9 Side B Stage 1   
Day 12      Order 9 Side A Stage 2   Order 9 Side A Stage 2   
Day 13      Order 0 Side A Stage 2   Order 0 Side A Stage 2   
Day 14      Order 4 Side B Stage 2   Order 4 Side B Stage 2   
Day 15     Order 13 Side B Stage 2  Order 13 Side B Stage 2   
Day 16                         NaN                      NaN   
Day 17                         NaN                      NaN   
Day 18     Order 12 Side A Stage 2  Order 12 Side A Stage 2   
Day 19      Order 7 Side B Stage 2   Order 7 Side B Stage 2   
Day 20      Order 3 Side A Stage 2   Order 3 Side A Stage 2   
Day 21     Order 11 Side A Stage 2  Order 11 Side A Stage 2   
Day 22      Order 6 Side A Stage 2   Order 6 Side A Stage 2   
Day 23                         NaN                      NaN   
Day 24                         NaN                      NaN   
Day 25      Order 5 Side A Stage 2   Order 5 Side A Stage 2   
Day 26                         NaN                      NaN   
Day 27                         NaN                      NaN   
Day 28                         NaN                      NaN   
Day 29                         NaN                      NaN   
Day 30                         NaN                      NaN   

Equipment              Equipment 1                           \
Bearing                  Bearing 2                Bearing 3   
Day 0       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 1       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 2                          NaN                      NaN   
Day 3                          NaN                      NaN   
Day 4       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 5       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 6       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 7       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 8       Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 9                          NaN                      NaN   
Day 10                         NaN                      NaN   
Day 11      Order 0 Side A Stage 1   Order 0 Side B Stage 1   
Day 12      Order 9 Side A Stage 2   Order 9 Side A Stage 2   
Day 13      Order 0 Side A Stage 2   Order 0 Side A Stage 2   
Day 14      Order 4 Side B Stage 2   Order 4 Side B Stage 2   
Day 15     Order 13 Side B Stage 2  Order 13 Side B Stage 2   
Day 16                         NaN                      NaN   
Day 17                         NaN                      NaN   
Day 18     Order 12 Side A Stage 2  Order 12 Side A Stage 2   
Day 19      Order 7 Side B Stage 2   Order 7 Side B Stage 2   
Day 20      Order 3 Side A Stage 2   Order 3 Side A Stage 2   
Day 21     Order 11 Side A Stage 2  Order 11 Side A Stage 2   
Day 22      Order 6 Side A Stage 2   Order 6 Side A Stage 2   
Day 23                         NaN                      NaN   
Day 24                         NaN                      NaN   
Day 25      Order 5 Side B Stage 2   Order 5 Side B Stage 2   
Day 26                         NaN                      NaN   
Day 27                         NaN                      NaN   
Day 28                        

In [26]:
# Replace day no. with date 
df.index = df.index.map(lambda x: (start_date + timedelta(days=int(x.split()[1]))).strftime('%m/%d'))

In [27]:
# Use color to represent different orders
color_pairs = {
    'Order 0': ('#1f77b4', '#aec7e8'),
    'Order 1': ('#ff7f0e', '#ffbb78'),
    'Order 2': ('#2ca02c', '#98df8a'),
    'Order 3': ('#d62728', '#ff9896'),
    'Order 4': ('#9467bd', '#c5b0d5'),
    'Order 5': ('#8c564b', '#c49c94'),
    'Order 6': ('#e377c2', '#f7b6d2'),
    'Order 7': ('#7f7f7f', '#c7c7c7'),
    'Order 8': ('#bcbd22', '#dbdb8d'),
    'Order 9': ('#17becf', '#9edae5'),
    'Order 10': ('#393b79', '#5254a3'),
    'Order 11': ('#637939', '#8ca252'),
    'Order 12': ('#8c6d31', '#bd9e39'),
    'Order 13': ('#843c39', '#ad494a'),
    'Order 14': ('#7b4173', '#a55194'),
    'Order 15': ('#17becf', '#98df8a'),
}

In [28]:
# Save the Excel
with pd.ExcelWriter(f"./results/result/{order_num}/scheduling_algorithm_order_{order_num}.xlsx") as writer:
    df.to_excel(writer, sheet_name="Schedule")
    
    # Access the xlsxwriter workbook and worksheet objects
    workbook  = writer.book
    worksheet = writer.sheets["Schedule"]
    
    # Set the width of each column
    for col_num, col in enumerate(df.columns, start=1):
        worksheet.set_column(col_num, col_num, 25)

    # set color for each order 
    for col_num, col in enumerate(df.columns, start=1):
        for row_num, value in enumerate(df[col], start=3):
            if pd.notna(value):
                # split the value, which is Order k Side i Stage j
                split_str = value.split()
                order_key = split_str[0] + " " + split_str[1]

                if "A" in split_str :
                    color = color_pairs[order_key][0]
                else:
                    color = color_pairs[order_key][1]
                          
                worksheet.write(row_num, col_num, value, workbook.add_format({'bg_color': color}))
